# DATA MATCHING AND CLEANING OF MATCHED DATA

---
Data matching in this chapter contains two sections


### 1) Data matching to generate coarse list of matched data (risk of duplicates data):
This is to show the matched data between T1 and T2. <br> List of matched data is obtained by matching the same subject, visit, and acquisition date for each group of AD, MCI, and CN. 


### 2) Matched data cleaning to generate list of final unique and balanced matched data (no duplicate)
This section is to clean the duplicated data found in matched data of T1 and T2 <br> There are also two steps here:<br><br>
<b>1. Removing duplicated data to tenerate cleaned list of matched data</b>  
    - To create the clean list of matched data, we remove the duplicated data from matched data list in T1 and T2<br>
    - The information and list of duplicated data is generated from `Metadata_Matched_Nolimit.xlsx` on section `duplicated_list`. The idea is to compare between availability of subjects from T1 and T2 list of cleaned matched data.
 
    
<b>2. Create unique and balanced matched data list</b> : <br>
    -  This is done by adding missed data for T1 and dropping the overflow data for T2<br>
    -  Since the duplicated values are all dropped in T1, there are some sets of data excluded in T1, but included in T2. 

In [2]:
import os
import nibabel as nib
from nibabel.testing import data_path
import numpy as np
import cv2 
from matplotlib import pyplot as plt 
import sys
import shutil
import pandas as pd
from volumentations import *

In [7]:
def getMetaDict(input_df):
    meta_dict = {} #dict
    # meta_dict[0].append(meta_df.loc[:,"Image Data ID"])
    meta_dict["ImageID"] = input_df.loc[:,"Image Data ID"]
    meta_dict["SubjectID"] = input_df.loc[:,"Subject"]
    meta_dict["Group"] = input_df.loc[:,"Group"]
    meta_dict["Sex"] = input_df.loc[:,"Sex"]
    meta_dict["Age"] = input_df.loc[:,"Age"]
    meta_dict["Visit"] = input_df.loc[:,"Visit"]
    meta_dict["Modality"] = input_df.loc[:,"Modality"]
    meta_dict["Description"] = input_df.loc[:,"Description"]
    meta_dict["Type"] = input_df.loc[:,"Type"]
    meta_dict["AcqData"] = input_df.loc[:,"Acq Date"]
    meta_dict["Format"] = input_df.loc[:,"Format"]
    return meta_dict

def getMetaString(input_df):
# meta_series = meta_df.loc[:,"Series.ID"]
# meta_study = meta_df.loc[:,"Study.ID"]
    meta_image = input_df.loc[:,"Image Data ID"]
    meta_subject = input_df.loc[:,"Subject"]
    meta_date = input_df.loc[:,"Acq Date"]
    meta_visit = input_df.loc[:,"Visit"]
    meta_group = input_df.loc[:,"Group"]
    
    meta_combined= []
    meta_combined.clear()
    imgIds = []
    imgIds.clear()
    i=0
    for i in range(0,len(meta_subject)):
        # combined = str(meta_subject[i])+"-"+str(meta_series[i])+"-"+str(meta_image[i])
        # if "CN" in meta_group[i]:
        #     # print("yes")
        #     meta_combined.append(meta_subject[i]+"-"+str(meta_visit[i])+"-"+str(meta_date[i]))
        # else:
        combined = str(meta_subject[i])+"-"+str(meta_visit[i])+"-"+str(meta_date[i])
        # combined = str(meta_visit[i]) 
        meta_combined.append(combined)
        imgIds.append(meta_image[i])
        i+=1
    return meta_combined, imgIds

def getIntersectFrame(input1, input2, seq="T1"):    
    imgArray = []
    imgArray.clear()
    string1, imgIds1  = getMetaString(input1)
    string2, imgIds2= getMetaString(input2)

    meta_dict = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    val = 0
    for i in range(len(string1)):    
        for j in range(len(string2)):
            if string1[i]==string2[j]: #Find the intersection
                # print("T1: {}, T2: {}".format(meta_image_1[i], meta_image_2[j])
                if "T1" in seq:
                    meta_dict["Image Data ID"].append(input1.loc[i,"Image Data ID"])
                    meta_dict["Subject"].append(input1.loc[i,"Subject"])
                    meta_dict["Group"].append(input1.loc[i,"Group"])
                    meta_dict["Sex"].append(input1.loc[i,"Sex"])
                    meta_dict["Age"].append(input1.loc[i,"Age"])
                    meta_dict["Visit"].append(input1.loc[i,"Visit"])
                    meta_dict["Modality"].append(input1.loc[i,"Modality"])
                    meta_dict["Description"].append(input1.loc[i,"Description"])
                    meta_dict["Type"].append(input1.loc[i,"Type"])
                    meta_dict["Acq Date"].append(input1.loc[i,"Acq Date"])
                    meta_dict["Format"].append( input1.loc[i,"Format"])

                # if "T1" in seq:
                #     # imgArray.append(string1[i].split("-")[0])
                #     imgArray.append(imgIds1[i])
                # else:
                    # imgArray.append(imgIds2[j])
                else:
                    meta_dict["Image Data ID"].append(input2.loc[j,"Image Data ID"])
                    meta_dict["Subject"].append(input2.loc[j,"Subject"])
                    meta_dict["Group"].append(input2.loc[j,"Group"])
                    meta_dict["Sex"].append(input2.loc[j,"Sex"])
                    meta_dict["Age"].append(input2.loc[j,"Age"])
                    meta_dict["Visit"].append(input2.loc[j,"Visit"])
                    meta_dict["Modality"].append(input2.loc[j,"Modality"])
                    meta_dict["Description"].append(input2.loc[j,"Description"])
                    meta_dict["Type"].append(input2.loc[j,"Type"])
                    meta_dict["Acq Date"].append(input2.loc[j,"Acq Date"])
                    meta_dict["Format"].append( input2.loc[j,"Format"])

                val+=1
                # print(val)
                # print(meta)
   
    # temp_meta_df = pd.DataFrame(meta_dict)  #convert the dict to a pandas DataFrame    
    # return imgArray
    # return meta_dict

        temp_meta_df = pd.DataFrame(meta_dict)  #convert the dict to a pandas DataFrame    
    return temp_meta_df

def exportMatchedCSV(input_df, seq, cond):
    meta_csv_file = "./TempMeta/Matched_Nolimit_{}w_{}_ADNI1_3T_12_05_2022.csv".format(seq, cond)
    # meta_csv_file = "./TempMeta/Matched_Nolimit_{}w_{}_ADNI1_3T_12_05_2022_Nonduplicate.csv".format(seq, cond) #updated in 01/02/2023 thinking about removing duplicate values in matched data csv

    with open(meta_csv_file, mode='w') as f:
        input_df.to_csv(f)

def exportCleanCSV(input_df, seq, cond):
    meta_csv_file = "./TempMeta/Cleaned_Matched_Nolimit_{}w_{}_ADNI1_3T_12_05_2022.csv".format(seq, cond)
    # meta_csv_file = "./TempMeta/Matched_Nolimit_{}w_{}_ADNI1_3T_12_05_2022_Nonduplicate.csv".format(seq, cond) #updated in 01/02/2023 thinking about removing duplicate values in matched data csv

    with open(meta_csv_file, mode='w') as f:
        input_df.to_csv(f)

def exportUniqueCSV(input_df, seq, cond):
    meta_csv_file = "./TempMeta/Unique_Matched_Nolimit_{}w_{}_ADNI1_3T_12_05_2022.csv".format(seq, cond)
    # meta_csv_file = "./TempMeta/Matched_Nolimit_{}w_{}_ADNI1_3T_12_05_2022_Nonduplicate.csv".format(seq, cond) #updated in 01/02/2023 thinking about removing duplicate values in matched data csv

    with open(meta_csv_file, mode='w') as f:
        input_df.to_csv(f)

# GET MATCHED DATA (COARSE LIST -> DUPLICATE RISK)

In [4]:
tesla = 3 #1.5 or 3
downloadDate = "12_05_2022"
# meta_path = "./Metadata/KHU_{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)
# path = "./{}T/{}/{}/".format(tesla, seq, cond)
seq = "T1"
cond= "AD" #condition is AD or CN or MCI
meta_path_1_ad = "./TempMeta/Cleaned_Ori_Nolimit_{}w_{}_ADNI1_{}T.csv".format(seq, cond, tesla, downloadDate)
cond= "MCI" #condition is AD or CN or MCI
meta_path_1_mci = "./TempMeta/Cleaned_Ori_Nolimit_{}w_{}_ADNI1_{}T.csv".format(seq, cond, tesla, downloadDate)
cond= "CN" #condition is AD or CN or MCI
meta_path_1_cn = "./TempMeta/Cleaned_Ori_Nolimit_{}w_{}_ADNI1_{}T.csv".format(seq, cond, tesla, downloadDate)
seq = "T2"
cond= "AD" #condition is AD or CN or MCI
meta_path_2_ad = "./TempMeta/Cleaned_Ori_Nolimit_{}w_{}_ADNI1_{}T.csv".format(seq, cond, tesla, downloadDate)
cond= "MCI" #condition is AD or CN or MCI
meta_path_2_mci = "./TempMeta/Cleaned_Ori_Nolimit_{}w_{}_ADNI1_{}T.csv".format(seq, cond, tesla, downloadDate)
cond= "CN" #condition is AD or CN or MCI
meta_path_2_cn = "./TempMeta/Cleaned_Ori_Nolimit_{}w_{}_ADNI1_{}T.csv".format(seq, cond, tesla, downloadDate)

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df_1_ad = pd.read_csv(meta_path_1_ad, delimiter=',')
meta_df_1_mci = pd.read_csv(meta_path_1_mci, delimiter=',')
meta_df_1_cn = pd.read_csv(meta_path_1_cn, delimiter=',')
meta_df_2_ad = pd.read_csv(meta_path_2_ad, delimiter=',')
meta_df_2_mci = pd.read_csv(meta_path_2_mci, delimiter=',')
meta_df_2_cn = pd.read_csv(meta_path_2_cn, delimiter=',')

# dict1 = getMetaDict(meta_df_1)
# dict2 = getMetaDict(meta_df_2)
# string1 = getMetaString(meta_df_1_ad)

inter_1_ad = getIntersectFrame(meta_df_1_ad, meta_df_2_ad, "T1")
inter_1_mci = getIntersectFrame(meta_df_1_mci, meta_df_2_mci, "T1")
inter_1_cn = getIntersectFrame(meta_df_1_cn, meta_df_2_cn, "T1")

inter_2_ad = getIntersectFrame(meta_df_1_ad, meta_df_2_ad, "T2")
inter_2_mci = getIntersectFrame(meta_df_1_mci, meta_df_2_mci, "T2")
inter_2_cn = getIntersectFrame(meta_df_1_cn, meta_df_2_cn, "T2")

# inter1 = inter_1_ad + inter_1_mci

# exportCSV(interdf1, "T1", cond)
# exportCSV(interdf2, "T2", cond)


20230204-160319


In [8]:
exportMatchedCSV(inter_1_ad, "T1", "AD")
exportMatchedCSV(inter_1_mci, "T1", "MCI")
exportMatchedCSV(inter_1_cn, "T1", "CN")
exportMatchedCSV(inter_2_ad, "T2", "AD")
exportMatchedCSV(inter_2_mci, "T2", "MCI")
exportMatchedCSV(inter_2_cn, "T2", "CN")


# CLEAN DUPLICATE AND BALANCE MATCHED DATA

### Clean duplicated data 

In [215]:
tesla = 3 #1.5 or 3
downloadDate = "12_05_2022"
# meta_path = "./Metadata/KHU_{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)
# path = "./{}T/{}/{}/".format(tesla, seq, cond)
seq = "T1"
cond= "AD" #condition is AD or CN or MCI
meta_path_1_ad = "./TempMeta/Matched_Nolimit_{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)
cond= "MCI" #condition is AD or CN or MCI
meta_path_1_mci = "./TempMeta/Matched_Nolimit_{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)
cond= "CN" #condition is AD or CN or MCI
meta_path_1_cn ="./TempMeta/Matched_Nolimit_{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)
seq = "T2"
cond= "AD" #condition is AD or CN or MCI
meta_path_2_ad = "./TempMeta/Matched_Nolimit_{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)
cond= "MCI" #condition is AD or CN or MCI
meta_path_2_mci = "./TempMeta/Matched_Nolimit_{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)
cond= "CN" #condition is AD or CN or MCI
meta_path_2_cn = "./TempMeta/Matched_Nolimit_{}w_{}_ADNI1_{}T_{}.csv".format(seq, cond, tesla, downloadDate)

In [216]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df_1_ad = pd.read_csv(meta_path_1_ad, delimiter=',')
meta_df_1_mci = pd.read_csv(meta_path_1_mci, delimiter=',')
meta_df_1_cn = pd.read_csv(meta_path_1_cn, delimiter=',')
meta_df_2_ad = pd.read_csv(meta_path_2_ad, delimiter=',')
meta_df_2_mci = pd.read_csv(meta_path_2_mci, delimiter=',')
meta_df_2_cn = pd.read_csv(meta_path_2_cn, delimiter=',')

# dict1 = getMetaDict(meta_df_1)
# dict2 = getMetaDict(meta_df_2)
# string1 = getMetaString(meta_df_1_ad)

inter_1_ad = getIntersectFrame(meta_df_1_ad, meta_df_2_ad, "T1")
inter_1_mci = getIntersectFrame(meta_df_1_mci, meta_df_2_mci, "T1")
inter_1_cn = getIntersectFrame(meta_df_1_cn, meta_df_2_cn, "T1")

inter_2_ad = getIntersectFrame(meta_df_2_ad, meta_df_2_ad, "T2")
inter_2_mci = getIntersectFrame(meta_df_1_mci, meta_df_2_mci, "T2")
inter_2_cn = getIntersectFrame(meta_df_1_cn, meta_df_2_cn, "T2")

# inter1 = inter_1_ad + inter_1_mci

# exportCSV(interdf1, "T1", cond)
# exportCSV(interdf2, "T2", cond)


20230203-022939


In [217]:
meta_df_1_ad

,Unnamed: 0,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format
0,0,I65946,067_S_1185,AD,M,63,3,MRI,MPRAGE,Original,8/07/2007,DCM
1,1,I65945,067_S_1185,AD,M,63,3,MRI,MPRAGE Repeat,Original,8/07/2007,DCM
2,2,I37523,032_S_1101,AD,F,71,2,MRI,MPRAGE,Original,1/25/2007,DCM
3,3,I55156,032_S_1101,AD,F,71,3,MRI,MPRAGE,Original,5/23/2007,DCM
4,4,I37524,032_S_1101,AD,F,71,2,MRI,MPRAGE Repeat,Original,1/25/2007,DCM
...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,I60917,131_S_0457,AD,F,84,4,MRI,MP-RAGE,Original,7/19/2007,DCM
96,96,I15182,136_S_0299,AD,F,89,2,MRI,MPRAGE repe,Original,5/12/2006,DCM
97,97,I29121,136_S_0299,AD,F,90,3,MRI,MPRAGE 3dtfe,Original,11/06/2006,DCM
98,98,I29120,136_S_0299,AD,F,90,3,MRI,MPRAGE repeat,Original,11/06/2006,DCM


In [ ]:
#-----Create clean list of matched data by dropping duplicated data

#---For T2, duplicated data can be noticed by the same value of Image ID
unique_df_2_ad = meta_df_2_ad.drop_duplicates(subset=["Image Data ID"])
unique_df_2_mci = meta_df_2_mci.drop_duplicates(subset=["Image Data ID"])
unique_df_2_cn = meta_df_2_cn.drop_duplicates(subset=["Image Data ID"])

#--For T1, Image ID is not the same value, but subject, sex, age, visit, and acq date
unique_df_1_ad = meta_df_1_ad.drop_duplicates(subset=["Subject", "Sex", "Age", "Visit", "Acq Date"], keep="last")
unique_df_1_mci = meta_df_1_mci.drop_duplicates(subset=["Subject", "Sex", "Age", "Visit", "Acq Date"], keep="last")
unique_df_1_cn = meta_df_1_cn.drop_duplicates(subset=["Subject", "Sex", "Age", "Visit", "Acq Date"], keep="last")

In [219]:
print(unique_df_2_ad.shape)
print(unique_df_2_mci.shape)
print(unique_df_2_cn.shape)
print(unique_df_1_ad.shape)
print(unique_df_1_mci.shape)
print(unique_df_1_cn.shape)


(50, 12)
(220, 12)
(153, 12)
(48, 12)
(206, 12)
(148, 12)


In [220]:
unique_df_2_ad = unique_df_2_ad.drop(columns=unique_df_2_ad.columns[0], axis=1)
unique_df_2_mci = unique_df_2_mci.drop(columns=unique_df_2_mci.columns[0], axis=1)
unique_df_2_cn = unique_df_2_cn.drop(columns=unique_df_2_cn.columns[0], axis=1)
unique_df_1_ad = unique_df_1_ad.drop(columns=unique_df_1_ad.columns[0], axis=1)
unique_df_1_mci = unique_df_1_mci.drop(columns=unique_df_1_mci.columns[0], axis=1)
unique_df_1_cn = unique_df_1_cn.drop(columns=unique_df_1_cn.columns[0], axis=1)

In [221]:
exportCleanCSV(unique_df_2_ad, "T2", "AD")
exportCleanCSV(unique_df_2_mci, "T2", "MCI")
exportCleanCSV(unique_df_2_cn, "T2", "CN")

exportCleanCSV(unique_df_1_ad, "T1", "AD")
exportCleanCSV(unique_df_1_mci, "T1", "MCI")
exportCleanCSV(unique_df_1_cn, "T1", "CN")

### Unique and Balanced List of Matched Data

In [222]:
def add2dataframe(added_list):
    #This function is to add the missed included data because of triple duplicate in T1
    meta_dict = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    i=0
    for i in range(len(added_list)):
        meta_dict["Image Data ID"].append(added_list[i].split("__")[0])
        meta_dict["Subject"].append(added_list[i].split("__")[1])
        meta_dict["Group"].append(added_list[i].split("__")[2])
        meta_dict["Sex"].append(added_list[i].split("__")[3])
        meta_dict["Age"].append(added_list[i].split("__")[4])
        meta_dict["Visit"].append(added_list[i].split("__")[5])
        meta_dict["Modality"].append(added_list[i].split("__")[6])
        meta_dict["Description"].append(added_list[i].split("__")[7])
        meta_dict["Type"].append(added_list[i].split("__")[8])
        meta_dict["Acq Date"].append(added_list[i].split("__")[9])
        meta_dict["Format"].append(added_list[i].split("__")[10])

        i=i+1
        
    temp_meta_df = pd.DataFrame(meta_dict)  #convert the dict to a pandas DataFrame    
    return temp_meta_df

def dropdataframe(df, dropped_list):
    i=0
    for i in range(len(dropped_list)):
        # print(dropped_list[i].split("__")[1])
        # temp_meta_df = df[df["Image Data ID"].str.contains(dropped_list[i].split("__")[1])==False]
        df = df[df["Image Data ID"].str.contains(dropped_list[i].split("__")[1])==False]
        i=i+1
    return df


---------------------------------------
Attention: At this moment

`add_1_{cond}` represents T1 data that was excluded due to triple duplicates.

The list of missed data obtained manually through excel with the filename `Metadata_Matched_Nolimit.xslx` by comparing 3 csv: `Cleaned_Matched_Nolimit` CSV of T1 and T2 from the same group, and `Matched_Nolimit` CSV of T1. The comparison result is stored in section `Duplicated_list`

The list of data information is also coppied manually by blocking the whole table and manually change the messy format to array initialization

---

In [223]:
# #---------------------------------------
# Attention: At this moment
# `add_1_{cond}` represents T1 data that was excluded due to triple duplicates.
# The list of missed data obtained manually through excel with the filename `Metadata_Matched_Nolimit.xslx` by comparing 3 csv: `Cleaned_Matched_Nolimit` CSV of T1 and T2 from the same group, and `Matched_Nolimit` CSV of T1. The comparison result is stored in section `Duplicated_list`
# The list of data information is also coppied manually by blocking the whole table and manually change the messy format to array initialization
#---------------------------------------


add_1_ad =["I29229__127_S_0844__AD__F__85__2__MRI__MP-RAGE REPEAT__Original__11/09/2006__DCM",
"I46444__130_S_1337__AD__M__72__2__MRI__MPRAGE SENSE repeat__Original__3/22/2007__DCM"]

add_1_cn = ["I17609__002_S_0559__CN__M__79__2__MRI__MPRAGE SENS__Original__6/27/2006__DCM",
"I38370__068_S_1191__CN__M__79__2___MRI__ADNI MPRAGE__Original__2/06/2007__DCM",
"I17176__126_S_0405__CN__M__76__2__MRI__MP-RAGE__Original__6/14/2006__DCM",
"I17639__126_S_0605__CN__F__76__2__MRI__MP-RAGE__Original__6/29/2006__DCM",
"I99235__128_S_1242__CN__F__72__4__MRI__MPRAGE Repeat__Original__3/21/2008__DCM"]

drop_2_mci = ["328__I78109__002_S_0954__MCI__F__70__4__MRI__Double_TSE__Original__10/17/2007__DCM",
"329__I78110__002_S_0954__MCI__F__70__4__MRI__Double_TSE__Original__10/17/2007__DCM",
"52__I99903__012_S_1292__MCI__M__77__4__MRI__DUAL_TSEad__Original__3/25/2008__DCM",
"66__I66877__016_S_1117__MCI__F__70__3__MRI__Axial PD-T2 TSE__Original__8/02/2007__DCM",
"114__I128637__031_S_1066__MCI__M__75__6__MRI__Axial PD-T2 TSE__Original__11/25/2008__DCM",
"290__I114314__051_S_1131__MCI__M__88__5__MRI__Axial PD-T2 TSE__Original__7/24/2008__DCM",
"86__I88901__068_S_0476__MCI__F__78__5__MRI__ADNI Double_TSE__Original__1/22/2008__DCM",
"87__I88900__068_S_0476__MCI__F__78__5__MRI__ADNI Double_TSE__Original__1/22/2008__DCM",
"88__I110244__068_S_0476__MCI__F__78__5__MRI__ADNI Double_TSE__Original__1/22/2008__DCM",
"424__I69857__130_S_0423__MCI__M__81__4__MRI__Double_TSE SENSE__Original__8/20/2007__DCM",
"426__I39428__130_S_0423__MCI__M__80__3__MRI__Double_TSE SENSE__Original__2/12/2007__DCM",
"15__I70956__130_S_0449__MCI__F__68__4__MRI__Double_TSE SENSE__Original__8/27/2007__DCM",
"16__I436758__130_S_0449__MCI__F__68__4__MRI__Double_TSE SENSE__Original__8/27/2007__DCM",
"102__I54996__131_S_0384__MCI__M__81__4__MRI__Axial PD/T2 FSE__Original__5/22/2007__DCM"]


add_df_1_ad = add2dataframe(add_1_ad)
unique_df_1_ad  = unique_df_1_ad.append(add_df_1_ad)
# unique_df_1_ad = unique_df_1_ad.drop(columns=unique_df_1_ad.columns[0], axis=1)

add_df_1_cn = add2dataframe(add_1_cn)
unique_df_1_cn = unique_df_1_cn.append(add_df_1_cn)
# unique_df_1_cn = unique_df_1_cn.drop(columns=unique_df_1_cn.columns[0], axis=1)

drop_df_2_mci = dropdataframe(unique_df_2_mci, drop_2_mci)
# unique_df_2_mci = unique_df_2_mci.drop(columns=unique_df_2_mci.columns[0], axis=1)

In [224]:
i=0
for i in range(len(drop_2_mci)):
    print(drop_2_mci[i].split("__")[1])
    # temp_meta_df = df[df["Image Data ID"].str.contains(dropped_list[i].split("__")[1])==False]
    unique_df_2_mci = unique_df_2_mci[unique_df_2_mci["Image Data ID"].str.contains(drop_2_mci[i].split("__")[1])==False]
    i=i+1
    

I78109
I78110
I99903
I66877
I128637
I114314
I88901
I88900
I110244
I69857
I39428
I70956
I436758
I54996


In [225]:
unique_df_2_mci

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format
0,I147843,005_S_0572,MCI,M,82,8,MRI,Axial PD/T2 FSE,Original,7/06/2009,DCM
1,I89321,005_S_0572,MCI,M,81,5,MRI,Axial PD/T2 FSE,Original,2/04/2008,DCM
2,I115281,005_S_0572,MCI,M,81,6,MRI,Axial PD/T2 FSE,Original,8/05/2008,DCM
3,I20025,005_S_0572,MCI,M,79,2,MRI,FSE PD/T2,Original,7/31/2006,DCM
4,I58877,005_S_0572,MCI,M,80,4,MRI,FSE PD/T2,Original,7/09/2007,DCM
...,...,...,...,...,...,...,...,...,...,...,...
452,I99400,136_S_0195,MCI,M,82,6,MRI,Double_TSE,Original,3/25/2008,DCM
453,I46775,136_S_0195,MCI,M,81,4,MRI,Double_TSE,Original,3/26/2007,DCM
454,I25483,136_S_0195,MCI,M,80,3,MRI,Double_TSE,Original,10/02/2006,DCM
455,I74927,136_S_0195,MCI,M,81,5,MRI,Double_TSE,Original,9/24/2007,DCM


In [226]:
print(unique_df_1_ad.shape)
print(unique_df_1_cn.shape)
print(unique_df_2_mci.shape)

(50, 11)
(153, 11)
(206, 11)


In [227]:
exportUniqueCSV(unique_df_2_ad, "T2", "AD")
exportUniqueCSV(unique_df_2_mci, "T2", "MCI")
exportUniqueCSV(unique_df_2_cn, "T2", "CN")

exportUniqueCSV(unique_df_1_ad, "T1", "AD")
exportUniqueCSV(unique_df_1_mci, "T1", "MCI")
exportUniqueCSV(unique_df_1_cn, "T1", "CN")